In [5]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [26]:
# group images from /data/Thermal Camera Images by class into folders

classes = ["NoGas", "Perfume", "Smoke", "Mixture"]
IMAGE_LENGTH = 480
IMAGE_WIDTH = 640

REORGANIZE_DATA = False

if REORGANIZE_DATA:
    for c in classes:
        if not os.path.exists("data/Thermal Camera Images/" + c):
            os.makedirs("data/Thermal Camera Images/" + c)

    # move images into folders
    # images follow the pattern: "#_class.png"

    # get all .png files in the directory
    files = os.listdir("data/Thermal Camera Images")
    png_files = [f for f in files if f.endswith(".png")]

    # move images into folders
    for f in png_files:
        # get the class of the image
        class_name = f.split("_")[1]
        # remove the .png extension
        class_name = class_name.split(".")[0]
        # move the image into the corresponding folder
        os.rename("data/Thermal Camera Images/" + f, "data/Thermal Camera Images/" + class_name + "/" + f)

In [48]:
class MultimodalDataset(tf.data.Dataset):
    def __init__(self, data, use_sensors=True, use_images=True):
        # filepaths = data['filepath']
        self.images = data['filepath'].values
        # MQ2 MQ3 MQ5 MQ6 MQ7 MQ8 MQ135
        self.sensor_data = data[['MQ2', 'MQ3', 'MQ5', 'MQ6', 'MQ7', 'MQ8', 'MQ135']].values
        # Gas values
        self.classes = data['Gas'].values
        # Gas one-hot encoded
        cat = OneHotEncoder()
        self.labels = cat.fit_transform(self.classes.reshape(-1, 1)).toarray()
        self.use_sensors = use_sensors
        self.use_images = use_images

        if not use_sensors and not use_images:
            raise ValueError("Both use_sensors and use_images cannot be False")
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        filepath = self.images[idx]
        image = tf.io.read_file(filepath)
        image = tf.image.decode_png(image, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        
        # normalize image
        image = image / 255.0

        sensor_data = self.sensor_data[idx]
        label = self.labels[idx]
        
        if self.use_sensors and self.use_images:
            return (image, sensor_data), label
        elif self.use_sensors:
            return sensor_data, label
        elif self.use_images:
            return image, label
        
    def _inputs(self):
        if self.use_images and self.use_sensors:
            return (tf.TensorSpec(shape=(None, IMAGE_LENGTH, IMAGE_WIDTH, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(None, 7), dtype=tf.float32))
        elif self.use_images:
            return tf.TensorSpec(shape=(None, IMAGE_LENGTH, IMAGE_WIDTH, 3), dtype=tf.float32)
        elif self.use_sensors:
            return tf.TensorSpec(shape=(None, 7), dtype=tf.float32)
        else:
            raise ValueError("At least one of use_images or use_sensors must be True")
        
    def element_spec(self):
        if self.use_images and self.use_sensors:
            return ((tf.TensorSpec(shape=(IMAGE_LENGTH, IMAGE_WIDTH, 3), dtype=tf.float32),
                     tf.TensorSpec(shape=(7,), dtype=tf.float32)),
                    tf.TensorSpec(shape=(), dtype=tf.int32))
        elif self.use_images:
            return (tf.TensorSpec(shape=(IMAGE_LENGTH, IMAGE_WIDTH, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(), dtype=tf.int32))
        elif self.use_sensors:
            return (tf.TensorSpec(shape=(7,), dtype=tf.float32),
                    tf.TensorSpec(shape=(), dtype=tf.int32))
        else:
            raise ValueError("At least one of use_images or use_sensors must be True")
        
    def get_path(self, idx):
        return self.images[idx]

In [49]:
# load sensor data from data/Gas Sensors Measurements/Gas_Sensor_Measurements.csv
file_data = pd.read_csv("data/Gas Sensors Measurements/Gas_Sensors_Measurements.csv")

# drop the first column
file_data = file_data.drop(columns=["Serial Number"])

# rename Corrosponding Image Name to filename
file_data = file_data.rename(columns={"Corresponding Image Name": "filepath"})

# change the image names from "#_class" to "/data/Thermal Camera Images/class/#_class.png"
file_data["filepath"] = file_data["filepath"].apply(lambda x: "data/Thermal Camera Images/" + x.split("_")[1] + "/" + x + ".png")

file_data.head()

,MQ2,MQ3,MQ5,MQ6,MQ7,MQ8,MQ135,Gas,filepath
0,555,515,377,338,666,451,416,NoGas,data/Thermal Camera Images/NoGas/0_NoGas.png
1,555,516,377,339,666,451,416,NoGas,data/Thermal Camera Images/NoGas/1_NoGas.png
2,556,517,376,337,666,451,416,NoGas,data/Thermal Camera Images/NoGas/2_NoGas.png
3,556,516,376,336,665,451,416,NoGas,data/Thermal Camera Images/NoGas/3_NoGas.png
4,556,516,376,337,665,451,416,NoGas,data/Thermal Camera Images/NoGas/4_NoGas.png


In [51]:
dataset = MultimodalDataset(file_data, use_sensors=True, use_images=True)

# get 10 random values from the dataset
for i in range(5):
    data, label = dataset[np.random.randint(0, len(dataset))]
    print(data[0].shape)
    print(data[1])
    # get the class name from the one-hot encoded label
    print(f'Class: {classes[np.argmax(label)]} label: {label}')
    print()

(640, 480, 3)
[740 530 389 383 556 549 451]
Class: Smoke label: [0. 0. 1. 0.]

(640, 480, 3)
[588 368 305 346 581 566 297]
Class: Mixture label: [0. 0. 0. 1.]

(640, 480, 3)
[767 532 450 457 641 719 489]
Class: Smoke label: [0. 0. 1. 0.]

(640, 480, 3)
[771 533 423 419 550 624 487]
Class: Perfume label: [0. 1. 0. 0.]

(640, 480, 3)
[772 530 432 437 596 671 485]
Class: Perfume label: [0. 1. 0. 0.]



In [14]:
# create an image only dataset from the first 100 images
image_dataset = MultimodalDataset(file_data[:100], use_sensors=False, use_images=True)

# create a simple CNN
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(480, 640, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
                metrics=['accuracy'])

history = model.fit(dataset, epochs=10, verbose=1)


2023-03-09 14:04:26.132033: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 278921216 exceeds 10% of free system memory.
2023-03-09 14:04:26.523957: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 278921216 exceeds 10% of free system memory.
2023-03-09 14:04:27.183637: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 278921216 exceeds 10% of free system memory.
/home/markbrub/.local/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10


2023-03-09 14:04:29.594853: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 278921216 exceeds 10% of free system memory.
2023-03-09 14:04:30.414959: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 278921216 exceeds 10% of free system memory.


: 

: 